In [1]:
from typing import Callable, Iterable, List
import os
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import numpy as np
import pandas as pd
from scipy import stats
from tqdm import tqdm

from lib.data import conditions as cond
from lib.data import tables as tab
from lib import bootstrap as bs

In [2]:
engine = tab.sa.create_engine(os.environ['SQLALCHEMY_ENGINE_URL'])
Session = tab.sa.orm.sessionmaker(bind=engine)
session = Session()

In [3]:
sample_mouseids = bs.get_mouseid_sampler(session, cond.CellType.pyr)
sample_cellids = {
    mid: lambda: bs.get_day_one_cell_ids(session, mid) 
    for mid in bs.get_mouseids_by_celltype(session, cond.CellType.pyr)
}
sample_averagetr = {
    day: {
        mid: bs.get_averagetrace_given_cellid_sampler(session, sample_cellids[mid], mid, day)
        for mid in bs.get_mouseids_by_celltype(session, cond.CellType.pyr)
    }
    for day in (1, 7)
}

In [4]:
for _ in range(10):
    mouseids = sample_mouseids()
    average_traces = []
    for mid in mouseids:
        average_traces.append(sample_averagetr[1][mid]())

<class 'lib.data.tables.SessionTrace'>


TypeError: 'SessionTrace' object is not subscriptable

In [6]:
sample_cellids['CL'()

TypeError: 'dict' object is not callable

In [4]:
sample_cellids['CL174']()

[945,
 946,
 947,
 948,
 949,
 950,
 951,
 952,
 953,
 954,
 955,
 956,
 957,
 958,
 959,
 960,
 961,
 962,
 963,
 964,
 965,
 966,
 967,
 968,
 969,
 970,
 971,
 972,
 973,
 974,
 975,
 976,
 977,
 978,
 979,
 980,
 981,
 982,
 983,
 984,
 985,
 986,
 987,
 988,
 989,
 990,
 991,
 992,
 993,
 994,
 995,
 996,
 997,
 998,
 999,
 1000,
 1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023,
 1024,
 1025,
 1026,
 1027,
 1028,
 1029,
 1030,
 1031,
 1032,
 1033,
 1034,
 1035,
 1036,
 1037,
 1038,
 1039,
 1040,
 1041,
 1042,
 1043,
 1044,
 1045,
 1046,
 1047,
 1048,
 1049,
 1050,
 1051,
 1052,
 1053,
 1054,
 1055,
 1056,
 1057,
 1058,
 1059,
 1060,
 1061,
 1062,
 1063,
 1064,
 1065,
 1066,
 1067,
 1068,
 1069,
 1070,
 1071,
 1072,
 1073,
 1074,
 1075,
 1076,
 1077,
 1078,
 1079,
 1080,
 1081,
 1082,
 1083,
 1084,
 1085,
 1086,
 1087,
 1088,
 1089,
 1090,
 1091,
 1092,
 1093,
 1094,
 1095

In [5]:
(
    session.query(tab.SessionTrace)
    .filter(tab.SessionTrace.day==1, tab.SessionTrace.cell_id==999)
    .first()
).trace

array([-7.8816150e+28, -1.3762600e+00,  1.2028697e-14, ...,
       -1.3949311e+00, -2.6179599e-03, -1.4392928e+00], dtype=float32)

In [11]:
dir(tab.SessionTrace)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__mapper__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__table__',
 '__tablename__',
 '__weakref__',
 '_decl_class_registry',
 '_sa_class_manager',
 'cell',
 'cell_id',
 'day',
 'metadata',
 'trace']